## IMDB 영화리뷰 감성분석

In [1]:
import numpy as np
import tensorflow as tf
seed = 2023
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
from tensorflow.keras.datasets import imdb
(X_train, y_train), (X_test, y_test) = imdb.load_data()

17464789/17464789 [==============================] - 0s 0us/step


In [3]:
len(X_train), len(X_test)

(25000, 25000)

In [4]:
# 등장한 단어수 (고유 단어수)
len(imdb.get_word_index())

1641221/1641221 [==============================] - 0s 0us/step


88584

In [5]:
print(X_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [6]:
# 리뷰 최대/평균 길이
max(len(s) for s in X_train), sum(len(s) for s in X_train) / len(X_train)

(2494, 238.71364)

#### 1. LSTM으로 감성분석
- 단어 빈도수 기준 많이 사용되는 10,000 단어만 사용
- 한 문장의 단어수 : 200

In [7]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
num_words = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

In [9]:
max_len = 200
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((25000, 200), (25000, 200))

- 모델
  - Embedding vector size: 100
  - LSTM node 수:128

In [10]:
model = Sequential([
    Embedding(num_words, 100, input_length=max_len),
    LSTM(128),    # activation=tahn 은 디폴트로 잡혀있어 생략가능
    Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          1000000   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1117377 (4.26 MB)
Trainable params: 1117377 (4.26 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
model.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best-imdb-lstm.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [12]:
hist = model.fit(
    X_train, y_train, validation_split=0.2,epochs=30, batch_size=64, callbacks=[mc,es]
)

Epoch 1/30
313/313 [==============================] - ETA: 0s - loss: 0.4824 - accuracy: 0.7571
Epoch 1: val_loss improved from inf to 0.41379, saving model to best-imdb-lstm.h5
313/313 [==============================] - 48s 130ms/step - loss: 0.4824 - accuracy: 0.7571 - val_loss: 0.4138 - val_accuracy: 0.8110
Epoch 2/30
  1/313 [..............................] - ETA: 4s - loss: 0.3438 - accuracy: 0.9062

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


313/313 [==============================] - ETA: 0s - loss: 0.2548 - accuracy: 0.8982
Epoch 2: val_loss improved from 0.41379 to 0.30218, saving model to best-imdb-lstm.h5
313/313 [==============================] - 18s 56ms/step - loss: 0.2548 - accuracy: 0.8982 - val_loss: 0.3022 - val_accuracy: 0.8778
Epoch 3/30
313/313 [==============================] - ETA: 0s - loss: 0.1769 - accuracy: 0.9360
Epoch 3: val_loss did not improve from 0.30218
313/313 [==============================] - 11s 34ms/step - loss: 0.1769 - accuracy: 0.9360 - val_loss: 0.3295 - val_accuracy: 0.8692
Epoch 4/30
313/313 [==============================] - ETA: 0s - loss: 0.1319 - accuracy: 0.9518
Epoch 4: val_loss did not improve from 0.30218
313/313 [==============================] - 7s 23ms/step - loss: 0.1319 - accuracy: 0.9518 - val_loss: 0.3684 - val_accuracy: 0.8676
Epoch 5/30
313/313 [==============================] - ETA: 0s - loss: 0.1885 - accuracy: 0.9199
Epoch 5: val_loss did not improve from 0.30218
31

In [13]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

782/782 [==============================] - 5s 5ms/step - loss: 0.3138 - accuracy: 0.8691


[0.31380847096443176, 0.8691200017929077]

##### 2. Conv1D로 학습

In [14]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout

In [17]:
model2 = Sequential([
    Embedding(num_words, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 7, activation='relu'),
    MaxPooling1D(7),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    GlobalMaxPooling1D(),
    Dense(1, 'sigmoid')
])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 100)          1000000   
                                                                 
 dropout (Dropout)           (None, 200, 100)          0         
                                                                 
 conv1d (Conv1D)             (None, 194, 64)           44864     
                                                                 
 max_pooling1d (MaxPooling1  (None, 27, 64)            0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 23, 64)            20544     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 4, 64)             0         
 g1D)                                                 

In [21]:
model2.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path2 = 'best-imdb-conv1d.h5'
mc2 = ModelCheckpoint(model_path2, save_best_only=True, verbose=1)
es2 = EarlyStopping(patience=5)

In [22]:
hist2 = model2.fit(
    X_train, y_train, validation_split=0.2,epochs=30, batch_size=64, callbacks=[mc2,es2]
)

Epoch 1/30
313/313 [==============================] - ETA: 0s - loss: 0.0289 - accuracy: 0.9894
Epoch 1: val_loss improved from inf to 0.80582, saving model to best-imdb-conv1d.h5
313/313 [==============================] - 26s 80ms/step - loss: 0.0289 - accuracy: 0.9894 - val_loss: 0.8058 - val_accuracy: 0.8384
Epoch 2/30
  3/313 [..............................] - ETA: 17s - loss: 0.0179 - accuracy: 0.9896

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


313/313 [==============================] - ETA: 0s - loss: 0.0127 - accuracy: 0.9958
Epoch 2: val_loss did not improve from 0.80582
313/313 [==============================] - 11s 35ms/step - loss: 0.0127 - accuracy: 0.9958 - val_loss: 1.0101 - val_accuracy: 0.8372
Epoch 3/30
313/313 [==============================] - ETA: 0s - loss: 0.0080 - accuracy: 0.9969
Epoch 3: val_loss did not improve from 0.80582
313/313 [==============================] - 6s 19ms/step - loss: 0.0080 - accuracy: 0.9969 - val_loss: 1.0610 - val_accuracy: 0.8388
Epoch 4/30
313/313 [==============================] - ETA: 0s - loss: 0.0107 - accuracy: 0.9959
Epoch 4: val_loss did not improve from 0.80582
313/313 [==============================] - 5s 15ms/step - loss: 0.0107 - accuracy: 0.9959 - val_loss: 1.2557 - val_accuracy: 0.8300
Epoch 5/30
313/313 [==============================] - ETA: 0s - loss: 0.0152 - accuracy: 0.9940
Epoch 5: val_loss did not improve from 0.80582
313/313 [==============================] -

In [23]:
best_model2 = load_model(model_path2)
best_model2.evaluate(X_test, y_test)

782/782 [==============================] - 2s 3ms/step - loss: 0.8912 - accuracy: 0.8133


[0.8912216424942017, 0.813319981098175]